In [16]:
import yfinance as yf
def ticker(symbol):
    t = yf.Ticker(symbol)
    return t
# documentation link:
# https://pypi.org/project/yfinance/

# approach sell at x% increase plus purchase and sell cost

from datetime import timedelta, date
import math
import pandas as pd


In [3]:
def ticker(symbol):
    t = yf.Ticker(symbol)
    return t

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [7]:
# retain working version 0.1

start_date = date(2020, 4, 28)
end_date = date.today()


for day in daterange(start_date, end_date):
    
    # print(day.strftime("%Y-%m-%d"))
    
    # get ticker information and history
    t = ticker('MSFT') # get stock ticker name
    h = t.history(start = day - timedelta(days = 5), end = day)
    
    # change in close
    h['Difference'] = h['Close'].diff(periods = 1)
    print(h)
    
h['Difference'] = h['Close'].diff(periods = 1)
h

              Open    High     Low   Close    Volume  Dividends  Stock Splits  \
Date                                                                            
2020-04-22  171.39  174.00  170.82  173.52  34651600          0             0   
2020-04-23  174.11  175.06  170.91  171.42  32790800          0             0   
2020-04-24  172.06  174.56  170.71  174.55  34305300          0             0   
2020-04-27  176.59  176.90  173.30  174.05  33194400          0             0   

            Difference  
Date                    
2020-04-22         NaN  
2020-04-23       -2.10  
2020-04-24        3.13  
2020-04-27       -0.50  
              Open    High     Low   Close    Volume  Dividends  Stock Splits  \
Date                                                                            
2020-04-23  174.11  175.06  170.91  171.42  32790800          0             0   
2020-04-24  172.06  174.56  170.71  174.55  34305300          0             0   
2020-04-27  176.59  176.90  173.30  174

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Difference
Date,,,,,,,,
2020-04-24,172.06,174.56,170.71,174.55,34305300,0,0,NaN
2020-04-27,176.59,176.90,173.30,174.05,33194400,0,0,-0.50
2020-04-28,175.59,175.67,169.39,169.81,34392700,0,0,-4.24
2020-04-29,173.22,177.68,171.88,177.43,51286600,0,0,7.62


In [81]:
# retain working version 0.5

start_date = date(2018, 2, 1)
end_date   = date.today()

ticker_name = 'MSFT'

# account value
cash = 1000

# init holdings
number_of_stocks = 0

# required return on holdings before sell-off
required_uptake  = 1.05 # 5 percent

# how many consecutive days down before we will buy (test for 1-5)
backlooking_scope = 5 # 1 en 2 werken niet vanwege weekenden en missende dagen

# ledger
df = pd.DataFrame()

for day in daterange(start_date, end_date):
    
    
    # get ticker information and history
    t = ticker(ticker_name) # get stock ticker name
    h = t.history(start = day - timedelta(days = backlooking_scope), end = day)
    
    # change in close
    h['Difference'] = h['Close'].diff(periods = 1)

    closing_price = h['Close'][-1]

    
    if any(h['Difference'] > 0) == False: # if any backlooking_score (5 days) is positive then this is true
        
        print(day.strftime("%Y-%m-%d") + ' buy')
        
#        closing_price = h['Close'][-1]
        
        if cash > closing_price: # SHOULD BE ACCOUNT VALUE 
            
            print('At the start your account value is {} and the stock goes at {}'.format(cash, closing_price))
            fixed_cost = 10
            number_of_stocks = math.floor((cash - fixed_cost) / (closing_price * 1.05))
                    #round(cash / (closing_price * 1.05), 0) # issue is that we can afford only our account value
            
            cost = fixed_cost + number_of_stocks * closing_price * 1.05
            cost_per_share = round(((cost / number_of_stocks) / closing_price) * closing_price,2)
            cash = round(cash - cost, 2) # watch out for infite and impossible selling number
            
            print('After purchase the account value is: {} and you have now {} stocks from MSTF @ ${}'.format(cash, number_of_stocks, closing_price))
            buy_date = day.strftime("%Y-%m-%d")

            # selling
            sell_price = (10 + number_of_stocks * closing_price * 1.025) * 1.05 - number_of_stocks * closing_price

            print('We will be selling when the price hits ${}'.format(cost_per_share))
            
            df = df.append({'order' : 'buy' , 'date' : buy_date, 'price' : closing_price, 
                            'volume' : number_of_stocks, 'acount_value' : cash},
                           ignore_index=True)

            
        else: # if backlooking_time (5 days) is negative we are not buying
            print('no funds available')
    
    else: # note that even on the 5 day down i might want to selse so shouldnt be a else
        print(day.strftime("%Y-%m-%d") + ' dont buy, current price {}'.format(closing_price))   
    
        if (number_of_stocks > 0) and (closing_price > cost_per_share):
            print('Sell, sell, sell {} stocks @ ${}'.format(number_of_stocks, cost_per_share))
            sell_date = day.strftime("%Y-%m-%d")
            
            cash = round(cash + number_of_stocks * cost_per_share) * 1.05
            print('You have sold {} stocks @ ${}'.format(number_of_stocks, cost_per_share))
            print('Youre account value is now: ${}'.format(cash ))
            # reset values
            number_of_stocks = 0
            cost_per_share = 0
            
            df = df.append({'order' : 'sell' , 'date' : sell_date, 'price' : closing_price, 
                            'volume' : number_of_stocks * -1, 'acount_value' : cash},
                           ignore_index=True)

        else:
            False

2018-02-01 dont buy, current price 91.78
2018-02-02 dont buy, current price 91.06
2018-02-03 dont buy, current price 88.66
2018-02-04 dont buy, current price 88.66
2018-02-05 dont buy, current price 88.66
2018-02-06 buy
At the start your account value is 1000 and the stock goes at 85.01
After purchase the account value is: 8.13 and you have now 11 stocks from MSTF @ $85.01
We will be selling when the price hits $90.17
2018-02-07 dont buy, current price 88.23
2018-02-08 dont buy, current price 86.57
2018-02-09 dont buy, current price 82.12
2018-02-10 dont buy, current price 85.18
2018-02-11 dont buy, current price 85.18
2018-02-12 dont buy, current price 85.18
2018-02-13 dont buy, current price 86.1
2018-02-14 dont buy, current price 86.78
2018-02-15 dont buy, current price 88.14
2018-02-16 dont buy, current price 89.93
2018-02-17 dont buy, current price 89.29
2018-02-18 dont buy, current price 89.29
2018-02-19 dont buy, current price 89.29
2018-02-20 dont buy, current price 89.29
2018-

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: divide by zero encountered in double_scalars


2018-03-28 dont buy, current price 86.84
2018-03-29 dont buy, current price 86.76
2018-03-30 dont buy, current price 88.58
2018-03-31 dont buy, current price 88.58
2018-04-01 dont buy, current price 88.58
2018-04-02 dont buy, current price 88.58
2018-04-03 dont buy, current price 85.91
2018-04-04 dont buy, current price 87.07
2018-04-05 dont buy, current price 89.61
2018-04-06 dont buy, current price 89.66
2018-04-07 dont buy, current price 87.57
2018-04-08 dont buy, current price 87.57
2018-04-09 dont buy, current price 87.57
2018-04-10 dont buy, current price 88.1
2018-04-11 dont buy, current price 90.15
2018-04-12 dont buy, current price 89.16
2018-04-13 dont buy, current price 90.82
2018-04-14 dont buy, current price 90.34
2018-04-15 dont buy, current price 90.34
2018-04-16 dont buy, current price 90.34
2018-04-17 dont buy, current price 91.4
2018-04-18 dont buy, current price 93.24
2018-04-19 dont buy, current price 93.6
2018-04-20 dont buy, current price 93.28
2018-04-21 dont buy

KeyboardInterrupt: 

In [63]:
# retain working version 0.6

start_date = date(2018, 2, 1)
end_date   = date.today()

# choose ticker
ticker_name = 'MSFT'

# account value
cash = 10000

# init holdings
number_of_stocks = 0

# required return on holdings before sell-off
required_uptake  = 1.05 # 5 percent

# how many consecutive days down before we will buy (test for 1-5)
scope = 5 # days # 1 en 2 werken niet vanwege weekenden en missende dagen

# ledger
df = pd.DataFrame()





for day in daterange(start_date, end_date):
    
    
    # get ticker information and history
    t = ticker(ticker_name) # get stock ticker name
    h = t.history(start = day - timedelta(days = scope), end = day)
    
    # add column with change in closing price
    h['Difference'] = h['Close'].diff(periods = 1)

    # current closing price
    closing_price = h['Close'][-1]
    
    if any(h['Difference'] > 0) == False: # buy, if x-days in a row the market trends down
        
        print(day.strftime("%Y-%m-%d") + ' buy')
        
#        closing_price = h['Close'][-1]
        
        if cash > closing_price: # SHOULD BE ACCOUNT VALUE 
            
            print('At the start your account value is {} and the stock goes at {}'.format(cash, closing_price))
            fixed_cost = 10
            number_of_stocks = math.floor((cash - fixed_cost) / (closing_price * 1.05))
                    #round(cash / (closing_price * 1.05), 0) # issue is that we can afford only our account value
            
            cost = fixed_cost + number_of_stocks * closing_price * 1.05
            cost_per_share = round(((cost / number_of_stocks) / closing_price) * closing_price,2)
            cash = round(cash - cost, 2) # watch out for infite and impossible selling number
            
            print('After purchase the account value is: {} and you have now {} stocks from MSTF @ ${}'.format(cash, number_of_stocks, closing_price))
            buy_date = day.strftime("%Y-%m-%d")

            # selling
            sell_price = (10 + number_of_stocks * closing_price * 1.025) * 1.05 - number_of_stocks * closing_price

            print('We will be selling when the price hits ${}'.format(cost_per_share))
            
            df = df.append({'order' : 'buy' , 'date' : buy_date, 'price' : closing_price, 
                            'volume' : number_of_stocks, 'acount_value' : cash},
                           ignore_index=True)

            
        else: # if scope (5 days) is negative we are not buying
            print('no funds available')
    
    else: #sell, if x-days in a row the market doesn't trend down
        
        print(day.strftime("%Y-%m-%d") + ' dont buy, current price {}'.format(closing_price))   
    
        if (number_of_stocks > 0) and (closing_price > cost_per_share):
            print('Sell, sell, sell {} stocks @ ${}'.format(number_of_stocks, cost_per_share))
            sell_date = day.strftime("%Y-%m-%d")
            
            cash = round(cash + number_of_stocks * cost_per_share) * 1.05
            print('You have sold {} stocks @ ${}'.format(number_of_stocks, cost_per_share))
            print('Youre account value is now: ${}'.format(cash ))
            # reset values
            number_of_stocks = 0
            cost_per_share = 0
            
            df = df.append({'order' : 'sell' , 'date' : sell_date, 'price' : closing_price, 
                            'volume' : number_of_stocks * -1, 'acount_value' : cash},
                           ignore_index=True)

        else:
            False

In [10]:

cash = 1000
fixed_cost = 10
number_of_stocks = math.floor((cash - fixed_cost) / (closing_price * 1.05))
                    #round(cash / (closing_price * 1.05), 0) # issue is that we can afford only our account value
            
cash = cash - (10 + number_of_stocks * closing_price * 1.05)

# Functions!

In [247]:
import yfinance as yf

def ticker(symbol):
    """ Return a ticker with relevant information from the yfinance api. """
    t = yf.Ticker(symbol)
    return t

def price(ticker_name, day):
    # how many consecutive days down before we will buy (test for 1-5)
    SCOPE = 5 # 1, 2 werken niet vanwege weekenden en missende dagen

    t = ticker(ticker_name)
    h = t.history(start = day - timedelta(days = SCOPE), end = day)
    closing_price = h['Close'][-1]
    
    return closing_price
    
    
def buy(ticker_name, day):
    """ For a given ticker, for a given day, check that ticker's history and return a buy / no-buy advice. """
    
    # how many consecutive days down before we will buy (test for 1-5)
    SCOPE = 5 # 1, 2 werken niet vanwege weekenden en missende dagen

    t = ticker(ticker_name)
    h = t.history(start = day - timedelta(days = SCOPE), end = day)
    price = h['Close'][-1]
    
    # add column with change in closing price
    h['Difference'] = h['Close'].diff(periods = 1)
    
    if any(h['Difference'] > 0) == False: # if down in last five days 
        buy = True
    else:
        buy = False
    return buy, price







def sell(ticker_name, no_stocks):
    
    if no_stocks <= 0:
        sell = False
    elif no_stocks > 0:
        sell = True
    return sell




def account(transaction_type, amount, account_value):
    
    if transaction_type == 'withdrawl':
        account_value = round(account_value - amount, 2)        
    elif transaction_type == 'deposit':
        account_value = round(account_value + amount, 2)
    else:
        print('Something went wrong in the account-function')

    return account_value


def purchase_power(account_value, p):
    """ Function to assess how many stocks I can buy.
    
    patches: don't invest everything into a single stock (10% of portfolio per transaction)
    """
    
    FIXED_COST = 7.90  
    VARIABLE_COST = 1.001 # 0.1%

    MARKUP = 1.05 # percentage needed markup before sale
    
    no_stocks = math.floor((account_value - FIXED_COST) / (p * VARIABLE_COST))
    
    if (no_stocks * p * VARIABLE_COST) < FIXED_COST:
        cost = FIXED_COST
    else:
        cost = VARIABLE_COST * no_stocks * p
    
    selling_price = round((cost / no_stocks) * MARKUP, 2)
        
    return no_stocks, cost, selling_price

In [251]:
start_date = date(2020, 5, 1)
end_date   = date.today()

stocks = 0
instruments = ['AAPL']
account_value = 1000
no_stocks = 0

df = pd.DataFrame()

for day in daterange(start_date, end_date):
    
    # real code, ran daily
    for instrument in instruments:

        print()
        b, p = buy(instrument, day) # boolean buy, price in dollar
    
        if   b == True:
            print('advice: buy, date: {}, for: {} @ ${}'.format(day, instrument, p)) 
      
            if (account_value >= 0) & (account_value > p):

                no_stocks, cost, selling_price = purchase_power(account_value, p)
                account_value = round(account('withdrawl', cost, account_value),2)
                
                print('----------------------------------------------------------------')
                print('On {}, purchase {} stocks from {} @ ${}'.format(day, no_stocks, instrument, p))
                print('Total cost ${}, gives account value after ${}'.format(cost, account_value))
                print('You will sell off these stocks at ${}'.format(selling_price))
                print('----------------------------------------------------------------')
                
                df = df.append({'order' : 'buy' , 'date' : day, 'price' : p, 
                            'volume' : no_stocks, 'acount_value' : selling_price}, ignore_index=True)

                
            else:
                print('You dont have enough money to buy, namely you have {}'.format(account_value))
                    
        elif b == False: # buy is false
            print("advice: don't buy, date: {}, for: {} @ ${}".format(day, instrument, p))
            
        else:
            print('Error, something went wrong in the buy-loop.')
            

            
    #for instrument in portoflio
            
            
#             s = sell(instrument, no_stocks)
        
#         if s == True:
            
#             #print(day, 'sell = true')
#             p = price(instrument, day)

#             if p > selling_price:
                
#                 FIXED_COST = 10
#                 VARIABLE_COST = 1.001
                
#                 print('Sell, sell, sell.')
#                 cost = FIXED_COST + (VARIABLE_COST * (no_stocks * p))
#                 gross_income = (no_stocks * p)
#                 netto_income = (no_stocks * p) - cost
#                 account_value = round(account('deposit', netto_income, account_value), 2)              
                                
#                 print('----------------------------------------------------------------')
#                 print('On {}, sell {} stocks from {} @ ${}'.format(day, no_stocks, instrument, p))
#                 print('Cost will be ${} on a gross sale of ${}'.format(cost, gross_income))
#                 print('Yield is ${}, gives account value after ${}'.format(netto_income, account_value))
#                 print('----------------------------------------------------------------')

                                            
#             else:
#                 print('Hold on a little longer buddy')
                
#         else:
#             False


advice: don't buy, date: 2020-05-01, for: AAPL @ $293.8

advice: don't buy, date: 2020-05-02, for: AAPL @ $289.07

advice: don't buy, date: 2020-05-03, for: AAPL @ $289.07


In [252]:
df

""


In [ ]:
df

In [4]:



def purchase_power(balance, price):
    
    FIXED_COST = 7.90
    VARIABLE_COST_PERC = 1.001
    MARKUP = 1.05
    
    
    variable_no_stocks = math.floor(balance / (price * variable_cost))
    fixed_no_stocks = math.floor((balance - fixed_cost) / price)
    no_stocks = np.where(fixed_no_stocks < variable_no_stocks, fixed_no_stocks, variable_no_stocks)
    
    for i in range(1, math.ceil(balance / price)):
        no_stocks = balance / (price * transaction_cost(i, p))
    
    
def fixed_cost_pp(balance, price):
    a = 1
    return a
    
def transaction_cost(no_stocks, price):
    """ Transaction cost based on fixed or variable cost."""
    
    FIXED_COST = 7.90
    VARIABLE_COST_PERC = 0.001
    
    if (no_stocks * price * VARIABLE_COST_PERC) > FIXED_COST:
        cost = no_stocks * price * VARIABLE_COST_PERC
    else:
        cost = FIXED_COST
    
    return cost

In [8]:
transaction_cost(600, 100)

60.0

In [43]:
def transaction_cost(no_stocks, price):
    """ Transaction cost based on fixed or variable cost."""

    FIXED_COST = 7.9
    VARIABLE_COST_PERC = 0.001
    
    if no_stocks * price * VARIABLE_COST_PERC < FIXED_COST:
        cost = round(FIXED_COST, 2)
    elif no_stocks * price * VARIABLE_COST_PERC > FIXED_COST:
        cost = round(no_stocks * price * VARIABLE_COST_PERC, 2)
    else:
        "Some error occured in transaction_cost function"
    
    return cost

In [45]:
transaction_cost(144, 147.12)

21.19

In [59]:
from sympy.solvers import solve
from sympy import Symbol
x = Symbol('x')
solve(x**2 - 1, x)

x = Symbol('x')

for b


solve( -1 * (price * i + transaction))


[-1, 1]

In [94]:
round(balance / price)

21

In [104]:
balance = 100
markup = 0.03
price_purchase = 10
fixed_cost = 7.9


# fixed cost
no_stocks = (balance - 2 * fixed_cost) / price_purchase
print(no_stocks)

breakeven = ((no_stocks * price_purchase + 2 * fixed_cost) / no_stocks) / (1 - markup)
print(breakeven)













8.42
12.243798516051621


In [93]:
BEP

balance = no_stocks * price_purchase + fixed_cost + fixed_cost

no_stocks * price_purchase + cost_purchase = no_stocks * price_sale + cost_sale + markup

                            + fixed_cost =                          + cost_sale = fixed_cost 
    
    
10 * 140 + 7.9 = 10 * x - 7.9
1407.9 + 7.9 = 10 * x
(1415.8 / 10) * x





total cost = 7.9 + 7.9 (cost_purchase + cost_sale)


for i in range(1, 10):
    print(i)
    
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
